In [0]:
df_hotel_reservation=spark.read.format("csv")\
                            .option("inferschema",True)\
                                .option("header",True)\
                                    .load("/FileStore/tables/Hotel_Reservation_Dataset.csv")

In [0]:
df_hotel_reservation.show()

+----------+------------+--------------+--------------------+-----------------+-----------------+------------------+---------+------------+-------------------+------------------+--------------+
|Booking_ID|no_of_adults|no_of_children|no_of_weekend_nights|no_of_week_nights|type_of_meal_plan|room_type_reserved|lead_time|arrival_date|market_segment_type|avg_price_per_room|booking_status|
+----------+------------+--------------+--------------------+-----------------+-----------------+------------------+---------+------------+-------------------+------------------+--------------+
|  INN00001|           2|             0|                   1|                2|      Meal Plan 1|       Room_Type 1|      224|  2017-10-02|            Offline|              65.0|  Not_Canceled|
|  INN00002|           2|             0|                   2|                3|     Not Selected|       Room_Type 1|        5|  2018-11-06|             Online|            106.68|  Not_Canceled|
|  INN00003|           1|     

In [0]:
df_hotel_reservation.columns

Out[3]: ['Booking_ID',
 'no_of_adults',
 'no_of_children',
 'no_of_weekend_nights',
 'no_of_week_nights',
 'type_of_meal_plan',
 'room_type_reserved',
 'lead_time',
 'arrival_date',
 'market_segment_type',
 'avg_price_per_room',
 'booking_status']

In [0]:
df_hotel_reservation.printSchema()

root
 |-- Booking_ID: string (nullable = true)
 |-- no_of_adults: integer (nullable = true)
 |-- no_of_children: integer (nullable = true)
 |-- no_of_weekend_nights: integer (nullable = true)
 |-- no_of_week_nights: integer (nullable = true)
 |-- type_of_meal_plan: string (nullable = true)
 |-- room_type_reserved: string (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- market_segment_type: string (nullable = true)
 |-- avg_price_per_room: double (nullable = true)
 |-- booking_status: string (nullable = true)



In [0]:
#What is the total number of reservations in the dataset?

df_hotel_reservation.count()

Out[5]: 700

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
#Which meal plan is the most popular among guests?

df_hotel_reservation.groupBy("type_of_meal_plan")\
                        .count().show(1)

+-----------------+-----+
|type_of_meal_plan|count|
+-----------------+-----+
|      Meal Plan 1|  527|
+-----------------+-----+
only showing top 1 row



In [0]:
#What is the average price per room for reservations involving children?

df_not_childern=df_hotel_reservation.filter(col("no_of_children")!=0)
df_not_childern.select(round(avg(col("avg_price_per_room")),2).alias("average_price")).show()


+-------------+
|average_price|
+-------------+
|       144.57|
+-------------+



In [0]:
#How many reservations were made for the year 20XX (replace XX with the desired year)?

df_hotel_reservation_2017=df_hotel_reservation.filter(year(col("arrival_date"))==2017)
df_hotel_reservation_2017.count()

Out[13]: 123

In [0]:
#What is the most commonly booked room type?

most_commom_room=df_hotel_reservation.groupBy("room_type_reserved").count()
most_commom_room.sort(col("count").desc()).show(1)

+------------------+-----+
|room_type_reserved|count|
+------------------+-----+
|       Room_Type 1|  534|
+------------------+-----+
only showing top 1 row



In [0]:
#How many reservations fall on a weekend (no_of_weekend_nights > 0)?

reservation_fall_on_weekend=df_hotel_reservation.filter(col("no_of_weekend_nights")>0)
reservation_fall_on_weekend.count()

Out[19]: 383

In [0]:
#What is the highest and lowest lead time for reservations?

df_hotel_reservation.select(max(col("lead_time")).alias("max_lead_time"),
                            min(col("lead_time")).alias("min_lead_time")).show()

+-------------+-------------+
|max_lead_time|min_lead_time|
+-------------+-------------+
|          443|            0|
+-------------+-------------+



In [0]:
#What is the most common market segment type for reservations?

common_market_seg=df_hotel_reservation.groupBy("market_segment_type").count()
common_market_seg.sort(col("count").desc()).show(1)

+-------------------+-----+
|market_segment_type|count|
+-------------------+-----+
|             Online|  518|
+-------------------+-----+
only showing top 1 row



In [0]:
#How many reservations have a booking status of "Confirmed"?

confirmed_booking=df_hotel_reservation.filter(col("booking_status")=="Not_Canceled")
confirmed_booking.count()

Out[25]: 493

In [0]:
#What is the total number of adults and children across all reservations?

df_hotel_reservation.select(sum(col("no_of_adults")).alias("total_adults"),
                            sum(col("no_of_children")).alias("total_childern")).show()

+------------+--------------+
|total_adults|total_childern|
+------------+--------------+
|        1316|            69|
+------------+--------------+



In [0]:
#What is the average number of weekend nights for reservations involving children?

resevatiopn_with_childrens=df_hotel_reservation.filter(col("no_of_children")>0)
resevatiopn_with_childrens.select(avg(col("no_of_weekend_nights")).alias("average_weekend")).show()

+---------------+
|average_weekend|
+---------------+
|            1.0|
+---------------+



In [0]:
#How many reservations were made in each month of the year?

month_yearwise_reservation=df_hotel_reservation.groupBy(year(col("arrival_date")).alias("year"),
                                                        month(col("arrival_date")).alias("month")).count()

month_yearwise_reservation.sort(col("year"),col("month")).show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2017|    7|    8|
|2017|    8|   14|
|2017|    9|   35|
|2017|   10|   40|
|2017|   11|   13|
|2017|   12|   13|
|2018|    1|   11|
|2018|    2|   28|
|2018|    3|   52|
|2018|    4|   67|
|2018|    5|   55|
|2018|    6|   84|
|2018|    7|   36|
|2018|    8|   56|
|2018|    9|   45|
|2018|   10|   63|
|2018|   11|   41|
|2018|   12|   39|
+----+-----+-----+



In [0]:
#What is the average number of nights (both weekend and weekday) spent by guests for each room type?

df_hotel_reservation.groupBy("room_type_reserved")\
                    .agg(round(avg("no_of_weekend_nights"),2).alias("avergae_weekend_nights"),
                         round(avg("no_of_week_nights"),2).alias("average_week_night")).show()
                        

+------------------+----------------------+------------------+
|room_type_reserved|avergae_weekend_nights|average_week_night|
+------------------+----------------------+------------------+
|       Room_Type 7|                   1.0|              1.67|
|       Room_Type 2|                   1.0|               2.0|
|       Room_Type 1|                  0.79|              2.09|
|       Room_Type 5|                   0.0|               2.5|
|       Room_Type 6|                  1.06|              2.56|
|       Room_Type 4|                  1.09|              2.71|
+------------------+----------------------+------------------+



In [0]:
#For reservations involving children, what is the most common room type, and what is the average price for that room type?

most_common_room_with_children=resevatiopn_with_childrens.groupBy("room_type_reserved")\
                            .agg(round(sum("avg_price_per_room"),2).alias("avergae_price"))

most_common_room_with_children.sort(col("avergae_price").desc()).show(1)

+------------------+-------------+
|room_type_reserved|avergae_price|
+------------------+-------------+
|       Room_Type 6|      3150.58|
+------------------+-------------+
only showing top 1 row



In [0]:
#Find the market segment type that generates the highest average price per room.

market_segment=df_hotel_reservation.groupBy("market_segment_type")\
                                    .agg(round(avg("avg_price_per_room"),2).alias("average_price"))

market_segment.sort(col("average_price").desc()).show(1)

+-------------------+-------------+
|market_segment_type|average_price|
+-------------------+-------------+
|             Online|       112.46|
+-------------------+-------------+
only showing top 1 row

